In [1]:
# --- 1. Imports and GPU Setup ---
import os, glob, random, numpy as np, tensorflow as tf
from collections import Counter
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

2025-05-14 11:08:15.449469: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# Optional: Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except Exception as e:
        print(f"GPU setup error: {e}")


2025-05-14 11:08:22.937575: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2025-05-14 11:08:23.151782: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:b7:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.39GiB deviceMemoryBandwidth: 1.41TiB/s
2025-05-14 11:08:23.151821: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2025-05-14 11:08:23.168912: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2025-05-14 11:08:23.168948: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2025-05-14 11:08:23.174768: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.

In [3]:
# --- 2. Data Generator ---
class NpzDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, file_paths, labels, batch_size=32, shuffle=True, target_size=(256, 256)):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.target_size = target_size
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_paths = self.file_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = [], []
        for p, label in zip(batch_paths, batch_labels):
            try:
                data = np.load(p)
                key = 'arr_0' if 'arr_0' in data else 'data' if 'data' in data else None
                if key is None:
                    print(f"⚠️ Skipping {p}: no 'arr_0' or 'data' key")
                    continue

                img = data[key]

                if img.ndim not in [2, 3]:
                    print(f"⚠️ Skipping {p}: 'images' must have 2 or 3 dimensions. Got shape: {img.shape}")
                    continue

                if img.ndim == 2:
                    img = np.expand_dims(img, axis=-1)

                if img.shape[:2] != self.target_size:
                    img = tf.image.resize(img, self.target_size).numpy()

                X.append(img.astype(np.float32) / 255.0)
                y.append(label)
            except Exception as e:
                print(f"⚠️ Skipping {p}: {str(e)}")
                continue
        if not X:
            return np.zeros((1, *self.target_size, 1), dtype=np.float32), np.zeros((1,), dtype=np.float32)
        return np.stack(X), np.array(y).astype(np.float32)

    def on_epoch_end(self):
        if self.shuffle:
            zipped = list(zip(self.file_paths, self.labels))
            random.shuffle(zipped)
            self.file_paths, self.labels = zip(*zipped)

In [4]:
# --- 3. Utility: Load paths and labels ---
def load_paths_and_labels(base_dir):
    files, labels = [], []
    for label in ['0', '1']:
        label_dir = os.path.join(base_dir, label)
        npz_files = glob.glob(os.path.join(label_dir, '*.npz'))
        files += npz_files
        labels += [int(label)] * len(npz_files)
    return files, labels


In [5]:
# --- 4. Prepare Data ---
train_dir = "/raid/mpsych/OMAMA/CLASSIFIER/1024_classifier/npz_split_1/train"
val_dir   = "/raid/mpsych/OMAMA/CLASSIFIER/1024_classifier/npz_split_1/val"
train_paths, train_labels = load_paths_and_labels(train_dir)
val_paths, val_labels     = load_paths_and_labels(val_dir)

In [6]:
# --- 5. Class Weights ---
counts = Counter(train_labels)
total = sum(counts.values())
class_weight = {0: total/(2*counts[0]), 1: total/(2*counts[1])}
print(f"Class weights: {class_weight}")


Class weights: {0: 0.5237414044069668, 1: 11.030126849894291}


In [7]:
# --- 6. Data Generators ---
BATCH_SIZE = 32
TARGET_SIZE = (256, 256)
train_gen = NpzDataGenerator(train_paths, train_labels, batch_size=BATCH_SIZE, target_size=TARGET_SIZE)
val_gen   = NpzDataGenerator(val_paths, val_labels, batch_size=BATCH_SIZE, target_size=TARGET_SIZE)


In [8]:
# --- 7. Model ---
def build_cnn(input_shape=(256,256,1)):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 5, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return models.Model(inputs, outputs)

model = build_cnn()
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      832       
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0     

2025-05-14 11:09:03.490723: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-14 11:09:03.494630: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:b7:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.39GiB deviceMemoryBandwidth: 1.41TiB/s
2025-05-14 11:09:03.501976: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2025-05-14 11:09:03.502357: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2025-05-14 11:09:04.368909: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect Stream

In [9]:
# --- 8. Callbacks ---
callbacks = [
    EarlyStopping(monitor='val_auc', patience=5, mode='max', restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_auc', mode='max'),
    ReduceLROnPlateau(monitor='val_auc', factor=0.5, patience=2, mode='max')
]

In [ ]:
# --- 9. Train ---
EPOCHS = 30
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weight,
    verbose=1
)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/30


2025-05-14 11:09:27.611712: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2025-05-14 11:09:27.632799: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2245855000 Hz
2025-05-14 11:09:29.205736: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2025-05-14 11:09:30.359249: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2025-05-14 11:09:31.974799: W tensorflow/stream_executor/gpu/asm_compiler.cc:191] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.0
2025-05-14 11:09:31.974827: W tensorflow/stream_executor/gpu/asm_compiler.cc:194] Used ptxas at ptxas
2025-05-14 11:09:31.975649: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modif

1422/3913 [=========>....................] - ETA: 26:04 - loss: 0.7101 - accuracy: 0.5111 - auc: 0.5656 - precision: 0.0532 - recall: 0.5798

In [ ]:
# --- 10. Evaluate ---
results = model.evaluate(val_gen, verbose=2)
print(f"Validation results: {dict(zip(model.metrics_names, results))}")